In [1]:
import os
import sys
import glob
import shutil
import warnings
from datetime import datetime
from timeit import default_timer as timer

In [2]:
RUN_ID = '2023-05-19_singles'

PROJECT_DIR = '/Users/research/projects/alderaan/'
RESULTS_DIR = os.path.join(PROJECT_DIR, 'Results/{0}/'.format(RUN_ID))
CATALOG     = os.path.join(PROJECT_DIR, 'Simulations/{0}/{0}.csv'.format(RUN_ID))

sys.path.append(PROJECT_DIR)
from alderaan.Results import Results

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'pymc3'

In [ ]:
truth = pd.read_csv(CATALOG)

In [ ]:
targets = glob.glob(RESULTS_DIR + '*')
targets.sort()

for i, t in enumerate(targets):
    targets[i] = t[-6:]

In [ ]:
results = []
ttv_type = []

for i, t in enumerate(targets):
    try:
        results.append(Results(t, RESULTS_DIR))
        
        use = np.array(truth.koi_id == 'K'+t[1:])
        npl = np.sum(use)
        for n in range(npl):
            ttv_type.append(np.array(truth[use].ttv_type)[n])
    except:
        pass

In [ ]:
keys = 'koi ror b T14'.split()
simulated = {}
posterior = {}

for k in keys:
    simulated[k] = []
    posterior[k] = []
    if k != 'koi':
        posterior[k+'_err'] = []

for i, r in enumerate(results):
    koi = 'K' + r.target[1:]
    use = truth.koi_id == koi
    npl = np.sum(use)
    
    for n in range(npl):
        simulated['koi'].append(np.array(truth[use].koi_id)[n])
        simulated['ror'].append(np.array(truth[use].ror)[n])
        simulated['b'].append(np.array(truth[use].impact)[n])
        simulated['T14'].append(np.array(truth[use].duration)[n])

        posterior['ror'].append(r.summary()['median']['ROR_{0}'.format(n)])
        posterior['b'].append(r.summary()['median']['IMPACT_{0}'.format(n)])
        posterior['T14'].append(r.summary()['median']['DUR14_{0}'.format(n)])
        
        posterior['ror_err'].append(r.summary()['stdev']['ROR_{0}'.format(n)])
        posterior['b_err'].append(r.summary()['stdev']['IMPACT_{0}'.format(n)])
        posterior['T14_err'].append(r.summary()['stdev']['DUR14_{0}'.format(n)])
        
for sk in simulated.keys():
    simulated[sk] = np.array(simulated[sk])
    
for pk in posterior.keys():
    posterior[pk] = np.array(posterior[pk])

In [ ]:
for k in keys:
    if k != 'koi':
        
        rel_err = (posterior[k]-simulated[k])/posterior[k+'_err']
        bad = (np.array(ttv_type) == 'gaussian')
        
        fig, ax = plt.subplots(1,2, figsize=(12,5))
        ax[0].errorbar(simulated[k], posterior[k], yerr=posterior[k+'_err'], fmt='k.')
        ax[0].errorbar(simulated[k][bad], posterior[k][bad], yerr=posterior[k+'_err'][bad], fmt='r.')        
        ax[0].set_title(k, fontsize=24)
        ax[0].set_xlabel('Truth', fontsize=20)
        ax[0].set_ylabel('Posterior', fontsize=20)
        ax[1].hist(rel_err[~bad], bins=np.arange(-4.5,5.,0.5))
        ax[1].set_title('Relative error', fontsize=20)
        plt.show()

In [ ]:
for i, r in enumerate(results):
    print(r.target)    
    r.plot_omc(show_outliers=True)